In [1]:
from urllib.request import urlopen # 인터넷 url를 열어주는 패키지
from urllib.parse import quote_plus # 한글을 유니코드 형식으로 변환해주는 코드 
from bs4 import BeautifulSoup # 
from selenium import webdriver # webdriver 가져오기
import time # 크롤링 중 시간 대기를 위한 패키지
import warnings # 경고메시지 제거 패키지
from selenium.webdriver.common.keys import Keys

warnings.filterwarnings(action='ignore') # 경고 메세지 제거

# 인스타 그램 url 생성
baseUrl = "https://www.instagram.com/explore/tags/"
plusUrl = input('검색할 태그를 입력하세요 : ')
url = baseUrl + quote_plus(plusUrl)

driver = webdriver.Chrome(
    executable_path="C:\\Users\Administrator\Downloads\\chromedriver.exe"
)
driver.get(url)


검색할 태그를 입력하세요 : 숭실대맛집추천


In [2]:
time.sleep(3)

# 로그인 하기
login_section = '//*[@id="react-root"]/section/nav/div[2]/div/div/div[3]/div/span/a[1]/button'
driver.find_element_by_xpath(login_section).click()
time.sleep(2)


elem_login = driver.find_element_by_name("username")
elem_login.clear()
elem_login.send_keys('ske0112@naver.com')

elem_login = driver.find_element_by_name('password')
elem_login.clear()
elem_login.send_keys('@1guddbs1588xx!')

time.sleep(1)

xpath = """//*[@id="react-root"]/section/main/div/article/div/div[1]/div/form/div/div[3]/button""" # 실제로는 여기까지가 블로그에서 첨부한 내용입니다.
driver.find_element_by_xpath(xpath).click()

time.sleep(4)

ypath = """//*[@id="react-root"]/section/main/div/div/div/div/button""" # 이 부분은 제가 직접 사용한 코드입니다.
driver.find_element_by_xpath(ypath).click()

In [3]:
!pip install lxml
!pip install tqdm

In [ ]:
from urllib.request import urlopen, Request
from tqdm import tqdm
import pandas as pd

In [ ]:
SCROLL_PAUSE_TIME = 1.0
reallink = []

while True:
    pageString = driver.page_source
    bsObj = BeautifulSoup(pageString, 'lxml')

    for link1 in bsObj.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
        for i in range(3):
            title = link1.select('a')[i]
            real = title.attrs['href']
            reallink.append(real)

    last_height = driver.execute_script('return document.body.scrollHeight')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        else:
            last_height = new_height
            continue

In [18]:
num_of_data = len(reallink)

print('총 {0}개의 데이터를 수집합니다.'.format(num_of_data))
csvtext = []

for i in tqdm(range(num_of_data)):

    csvtext.append([])
    req = Request("https://www.instagram.com/p"+reallink[i], headers={'User-Agent': 'Mozila/5.0'})

    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'lxml', from_encoding='utf-8')
    soup1 = soup.find('meta', attrs={'property':"og:description"})

    reallink1 = soup1['content']
    reallink1 = reallink1[reallink1.find("@") + 1:reallink1.find(")")]
    reallink1 = reallink1[:20]

    if reallink1 == '':
        reallink1 = "Null"
    csvtext[i].append(reallink1)

    for reallink2 in soup.find_all('meta', attrs={'property':"instapp:hashtags"}):
        hashtags = reallink2['content'].rstrip(',')
        csvtext[i].append(hashtags)

    # csv로 저장

    data = pd.DataFrame(csvtext)
    data.to_csv('insta2.txt', encoding='utf-8')

driver.close()

  0%|                                                  | 0/374 [00:00<?, ?it/s]

총 374개의 데이터를 수집합니다.


100%|████████████████████████████████████████| 374/374 [06:48<00:00,  1.09s/it]


InvalidSessionIdException: Message: invalid session id


In [25]:
path = 'C:\\Users\Administrator\Desktop\비즈니스_애널리틱스\\insta2.txt'

insta = pd.read_csv(path, encoding = 'utf-8')
insta.drop('Unnamed: 0', axis = 1, inplace = True)


In [26]:
insta.head()
insta

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,gwanak_nolja,상도동카페,숭실대술집,상도동데이트코스,상도동데이트,숭실대술집추천,먹스타맞팔,숭실대데이트,먹부림,숭실대카페,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gwanak_nolja,흑석동술집추천,흑석동카페,흑석동맛집추천,숭실대맛집,중앙대카페,흑석동술집,흑석동핫플레이스,중앙대맛집,중앙대맛집추천,...,중앙대데이트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,gwanak_nolja: “중앙대 인,상도동핫플레이스,숭실대맛집추천,상도동술집추천,상도동맛집,상도동데이트,베스트모음,숭실대핫플,숭실대데이트,동작구맛집,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,gwanak_nolja,상도동핫플,숭실대맛집,숭실대맛집추천,숭실대술집,상도동데이트,상도동핫플레이스,상도동술집,상도동카페,베스트모음,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Null,숭실대술집,숭실대데이트,숭실대카페,중앙대데이트,중앙대술집,숭실대맛집추천,중앙대카페,숭실대맛집,중앙대맛집,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,eunah.ryu,인생감자탕,감자탕맛집,번창하세요,배부르지만먹는다,볶음밥사랑,숭실대맛집추천,상도동인가,위는위대하다,아이갓잇🤓,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,copspa22 on Instagra,숭실대맛집추천,숭실대입구역맛집,숭실대맛집,숭실대밥집,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
371,서산먹부림 1987 토끼주방 미라뷰의,상도역초밥맛집,중앙대맛집,숭실대초밥,초밥은역시,상도역초밥,상도맛집,상도역고우마구로,숭실대맛집추천,중앙대초밥,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372,donggi4468,수제돈까스맛집,노량진맛집추천,상도동맛집추천,숭실대맛집추천,김과장수제즉석돈까스,대방동맛집추천,돈까스맛집추천,중앙대학교맛집추천,장승배기맛집추천,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
insta.index

RangeIndex(start=0, stop=374, step=1)

In [68]:
insta_df = pd.read_csv('C:\\Users\Administrator\Desktop\비즈니스_애널리틱스\\insta2.txt')
insta_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,21,22,23,24,25,26,27,28,29,30
0,0,gwanak_nolja,상도동카페,숭실대술집,상도동데이트코스,상도동데이트,숭실대술집추천,먹스타맞팔,숭실대데이트,먹부림,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,gwanak_nolja,흑석동술집추천,흑석동카페,흑석동맛집추천,숭실대맛집,중앙대카페,흑석동술집,흑석동핫플레이스,중앙대맛집,...,중앙대데이트,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,gwanak_nolja: “중앙대 인,상도동핫플레이스,숭실대맛집추천,상도동술집추천,상도동맛집,상도동데이트,베스트모음,숭실대핫플,숭실대데이트,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,gwanak_nolja,상도동핫플,숭실대맛집,숭실대맛집추천,숭실대술집,상도동데이트,상도동핫플레이스,상도동술집,상도동카페,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Null,숭실대술집,숭실대데이트,숭실대카페,중앙대데이트,중앙대술집,숭실대맛집추천,중앙대카페,숭실대맛집,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,369,eunah.ryu,인생감자탕,감자탕맛집,번창하세요,배부르지만먹는다,볶음밥사랑,숭실대맛집추천,상도동인가,위는위대하다,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,370,copspa22 on Instagra,숭실대맛집추천,숭실대입구역맛집,숭실대맛집,숭실대밥집,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
371,371,서산먹부림 1987 토끼주방 미라뷰의,상도역초밥맛집,중앙대맛집,숭실대초밥,초밥은역시,상도역초밥,상도맛집,상도역고우마구로,숭실대맛집추천,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372,372,donggi4468,수제돈까스맛집,노량진맛집추천,상도동맛집추천,숭실대맛집추천,김과장수제즉석돈까스,대방동맛집추천,돈까스맛집추천,중앙대학교맛집추천,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
insta_df = insta_df.loc[:,'1':'10']
insta_df

,1,2,3,4,5,6,7,8,9,10
0,상도동카페,숭실대술집,상도동데이트코스,상도동데이트,숭실대술집추천,먹스타맞팔,숭실대데이트,먹부림,숭실대카페,먹스타그램
1,흑석동술집추천,흑석동카페,흑석동맛집추천,숭실대맛집,중앙대카페,흑석동술집,흑석동핫플레이스,중앙대맛집,중앙대맛집추천,숭실대핫플
2,상도동핫플레이스,숭실대맛집추천,상도동술집추천,상도동맛집,상도동데이트,베스트모음,숭실대핫플,숭실대데이트,동작구맛집,동작구핫플
3,상도동핫플,숭실대맛집,숭실대맛집추천,숭실대술집,상도동데이트,상도동핫플레이스,상도동술집,상도동카페,베스트모음,상도동맛집추천
4,숭실대술집,숭실대데이트,숭실대카페,중앙대데이트,중앙대술집,숭실대맛집추천,중앙대카페,숭실대맛집,중앙대맛집,중앙대핫플
...,...,...,...,...,...,...,...,...,...,...
369,인생감자탕,감자탕맛집,번창하세요,배부르지만먹는다,볶음밥사랑,숭실대맛집추천,상도동인가,위는위대하다,아이갓잇🤓,해장국맛집
370,숭실대맛집추천,숭실대입구역맛집,숭실대맛집,숭실대밥집,NaN,NaN,NaN,NaN,NaN,NaN
371,상도역초밥맛집,중앙대맛집,숭실대초밥,초밥은역시,상도역초밥,상도맛집,상도역고우마구로,숭실대맛집추천,중앙대초밥,중앙대맛집추천
372,수제돈까스맛집,노량진맛집추천,상도동맛집추천,숭실대맛집추천,김과장수제즉석돈까스,대방동맛집추천,돈까스맛집추천,중앙대학교맛집추천,장승배기맛집추천,NaN


In [70]:
insta_df.describe()

,1,2,3,4,5,6,7,8,9,10
count,340,327,326,319,304,288,272,262,250,247
unique,163,164,164,161,162,166,153,142,135,143
top,숭실대맛집추천,숭실대맛집,숭실대맛집,숭실대맛집,숭실대맛집,숭실대맛집추천,숭실대맛집추천,숭실대맛집추천,숭실대맛집,숭실대맛집
freq,35,27,30,26,26,26,16,22,15,15


In [89]:
insta_df = insta_df.drop_duplicates()  # 중복되는 값 제거
insta_df = insta_df.dropna(axis=0)     # 결측값이 있는 열 제거
insta_df

,1,2,3,4,5,6,7,8,9,10
0,상도동카페,숭실대술집,상도동데이트코스,상도동데이트,숭실대술집추천,먹스타맞팔,숭실대데이트,먹부림,숭실대카페,먹스타그램
1,흑석동술집추천,흑석동카페,흑석동맛집추천,숭실대맛집,중앙대카페,흑석동술집,흑석동핫플레이스,중앙대맛집,중앙대맛집추천,숭실대핫플
2,상도동핫플레이스,숭실대맛집추천,상도동술집추천,상도동맛집,상도동데이트,베스트모음,숭실대핫플,숭실대데이트,동작구맛집,동작구핫플
4,숭실대술집,숭실대데이트,숭실대카페,중앙대데이트,중앙대술집,숭실대맛집추천,중앙대카페,숭실대맛집,중앙대맛집,중앙대핫플
8,숭실대술집추천,상도동핫플레이스,숭실대핫플,숭실대데이트,베스트모음,숭실대카페,관악동작구로금천놀자,상도동맛집추천,상도동핫플,중앙대맛집
12,먹스타맞팔환영,서울맛집추천,foodstagram,서울맛집,l4l,먹스타,노또먹,먹스타맞팔,닭갈비,코웍비스트로
60,중앙대맛집,서울와인바추천,와인바,좋반,숭실대맛집,숭실대역맛집,숭실대맛집추천,상도움프바,밀감_상도🍊,서울와인바
77,오르간,오르간악보,관악구맛집,게이트랙,관악구음악,서울대입구맛집추천,빈티지,쿡스나인커피,앤틱,앤틱소품
105,숭실대맛집,연탄구이,숭실대삼겹살,웨이팅맛집,불금시작,삼겹살맛집,숭실대고깃집,불타는소금구이,고기스타그램,숭실대고기집
116,피아노협주곡,봄봄봄,피아노,서울대맛집,관악구맛집,오르간악보,넉넉한손소독제도준비완료,게이트랙,고속터미널꽃시장,오르간


In [134]:
insta_df = insta_df.loc[insta_df['1'].str.contains('숭실대') == False]
insta_df

,1,2,3,4,5,6,7,8,9,10
0,상도동카페,숭실대술집,상도동데이트코스,상도동데이트,숭실대술집추천,먹스타맞팔,숭실대데이트,먹부림,숭실대카페,먹스타그램
1,흑석동술집추천,흑석동카페,흑석동맛집추천,숭실대맛집,중앙대카페,흑석동술집,흑석동핫플레이스,중앙대맛집,중앙대맛집추천,숭실대핫플
2,상도동핫플레이스,숭실대맛집추천,상도동술집추천,상도동맛집,상도동데이트,베스트모음,숭실대핫플,숭실대데이트,동작구맛집,동작구핫플
12,먹스타맞팔환영,서울맛집추천,foodstagram,서울맛집,l4l,먹스타,노또먹,먹스타맞팔,닭갈비,코웍비스트로
60,중앙대맛집,서울와인바추천,와인바,좋반,숭실대맛집,숭실대역맛집,숭실대맛집추천,상도움프바,밀감_상도🍊,서울와인바
77,오르간,오르간악보,관악구맛집,게이트랙,관악구음악,서울대입구맛집추천,빈티지,쿡스나인커피,앤틱,앤틱소품
116,피아노협주곡,봄봄봄,피아노,서울대맛집,관악구맛집,오르간악보,넉넉한손소독제도준비완료,게이트랙,고속터미널꽃시장,오르간
117,쌀국수맛집,상도맛집추천,숭실대고렝,상도맛집,고렝,베트남음식,태국음식,대만음식,숭실대,팟타이
120,양념치킨,콜라겐덩어리,ᴊᴍᴛɢʀ,헬요일,헬요일을대하는자세,치킨맛집,동작구맛집추천,간장치킨,후라이드치킨,꼭가세요
155,관악구맛집,서울대입구맛집추천,쿡스나인커피,관악구음악,게이트랙,고속터미널꽃시장,앤틱,연주회선물,봄봄봄,오르간악보


In [141]:
insta_df = insta_df.loc[insta_df['1'].str.contains('상도동') == False]
insta_df

,1,2,3,4,5,6,7,8,9,10
1,흑석동술집추천,흑석동카페,흑석동맛집추천,숭실대맛집,중앙대카페,흑석동술집,흑석동핫플레이스,중앙대맛집,중앙대맛집추천,숭실대핫플
12,먹스타맞팔환영,서울맛집추천,foodstagram,서울맛집,l4l,먹스타,노또먹,먹스타맞팔,닭갈비,코웍비스트로
60,중앙대맛집,서울와인바추천,와인바,좋반,숭실대맛집,숭실대역맛집,숭실대맛집추천,상도움프바,밀감_상도🍊,서울와인바
77,오르간,오르간악보,관악구맛집,게이트랙,관악구음악,서울대입구맛집추천,빈티지,쿡스나인커피,앤틱,앤틱소품
116,피아노협주곡,봄봄봄,피아노,서울대맛집,관악구맛집,오르간악보,넉넉한손소독제도준비완료,게이트랙,고속터미널꽃시장,오르간
117,쌀국수맛집,상도맛집추천,숭실대고렝,상도맛집,고렝,베트남음식,태국음식,대만음식,숭실대,팟타이
120,양념치킨,콜라겐덩어리,ᴊᴍᴛɢʀ,헬요일,헬요일을대하는자세,치킨맛집,동작구맛집추천,간장치킨,후라이드치킨,꼭가세요
155,관악구맛집,서울대입구맛집추천,쿡스나인커피,관악구음악,게이트랙,고속터미널꽃시장,앤틱,연주회선물,봄봄봄,오르간악보
165,선팔,숭실대레스토랑,선팔환영,숭실대맛집추천,foodporn,foodie,instafood,로제파스타,숭실대맛집,fff
198,ᴊᴍᴛɢʀ,양념치킨,또먹어,오늘먹은거아님,먹스타그램🐷,숭실대먹을만한곳,치킨맛집,서비스,닭발존맛,맛스타그램촵촵


In [151]:
insta_df = insta_df.loc[insta_df['2'].str.contains('상도') == False]
insta_df = insta_df.loc[insta_df['3'].str.contains('상도') == False]
insta_df = insta_df.loc[insta_df['3'].str.contains('상도') == False]
insta_df = insta_df.loc[insta_df['3'].str.contains('상도') == False]

insta_df = insta_df.loc[insta_df['3'].str.contains('상도') == False]



In [153]:
insta_df = insta_df.loc[insta_df['3'].str.contains('숭실') == False]
insta_df = insta_df.loc[insta_df['4'].str.contains('숭실') == False]
insta_df = insta_df.loc[insta_df['5'].str.contains('숭실') == False]
insta_df = insta_df.loc[insta_df['6'].str.contains('숭실') == False]
insta_df = insta_df.loc[insta_df['7'].str.contains('숭실') == False]

In [155]:
insta_df = insta_df.loc[insta_df['3'].str.contains('흑석') == False]
insta_df = insta_df.loc[insta_df['4'].str.contains('흑석') == False]
insta_df = insta_df.loc[insta_df['5'].str.contains('흑석') == False]
insta_df = insta_df.loc[insta_df['6'].str.contains('흑석') == False]
insta_df = insta_df.loc[insta_df['7'].str.contains('흑석') == False]
insta_df

,1,2,3,4,5,6,7,8,9,10
12,먹스타맞팔환영,서울맛집추천,foodstagram,서울맛집,l4l,먹스타,노또먹,먹스타맞팔,닭갈비,코웍비스트로
77,오르간,오르간악보,관악구맛집,게이트랙,관악구음악,서울대입구맛집추천,빈티지,쿡스나인커피,앤틱,앤틱소품
116,피아노협주곡,봄봄봄,피아노,서울대맛집,관악구맛집,오르간악보,넉넉한손소독제도준비완료,게이트랙,고속터미널꽃시장,오르간
120,양념치킨,콜라겐덩어리,ᴊᴍᴛɢʀ,헬요일,헬요일을대하는자세,치킨맛집,동작구맛집추천,간장치킨,후라이드치킨,꼭가세요
155,관악구맛집,서울대입구맛집추천,쿡스나인커피,관악구음악,게이트랙,고속터미널꽃시장,앤틱,연주회선물,봄봄봄,오르간악보
330,배부르지만먹는다,해장국맛집,아이갓잇🤓,라면사리,인생감자탕,뼈추가는기본,상도동인가,번창하세요,볶음밥사랑,감자탕에


In [160]:


insta_df = insta_df.loc[insta_df['4'].str.contains('서울') == False]
insta_df = insta_df.loc[insta_df['5'].str.contains('서울') == False]
insta_df = insta_df.loc[insta_df['6'].str.contains('서울') == False]
insta_df = insta_df.loc[insta_df['7'].str.contains('서울') == False]
insta_df

,1,2,3,4,5,6,7,8,9,10
120,양념치킨,콜라겐덩어리,ᴊᴍᴛɢʀ,헬요일,헬요일을대하는자세,치킨맛집,동작구맛집추천,간장치킨,후라이드치킨,꼭가세요
330,배부르지만먹는다,해장국맛집,아이갓잇🤓,라면사리,인생감자탕,뼈추가는기본,상도동인가,번창하세요,볶음밥사랑,감자탕에


In [162]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [163]:
!pip install scikit-learn
!pip install scipy

In [164]:
tfidf_vectorizer = TfidfVectorizer(min_df=1)

In [165]:
tfidf_matrix = tfidf_vectorizer.fit_transform(insta_df)

In [166]:
document_distances = (tfidf_matrix * tfidf_matrix.T)

In [167]:
print(document_distances.toarray())

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
